# TensorFlow 2.0+ Low Level APIs Convert Example

This example demonstrates the workflow to build a model using
TensorFlow 2.0+ low-level APIs and convert it to Core ML 
`.mlmodel` format using the `coremltools.converters.tensorflow` converter.
For more example, refer `test_tf_2x.py` file.

Note: 

- This notebook was tested with following dependencies:

```
tensorflow==2.0.0
coremltools==3.1
```

- Models from TensorFlow 2.0+ is supported only for `minimum_ios_deployment_target>=13`.
You can also use `tfcoreml.convert()` instead of 
`coremltools.converters.tensorflow.convert()` to convert your model.

In [1]:
import tensorflow as tf
import numpy as np
import coremltools

print(tf.__version__)
print(coremltools.__version__)

W1101 14:02:33.174557 4762860864 __init__.py:74] TensorFlow version 2.0.0 detected. Last version known to be fully compatible is 1.14.0 .


2.0.0
3.1


## Using Low-Level APIs

In [2]:
# construct a toy model with low level APIs
root = tf.train.Checkpoint()
root.v1 = tf.Variable(3.)
root.v2 = tf.Variable(2.)
root.f = tf.function(lambda x: root.v1 * root.v2 * x)

# save the model
saved_model_dir = './tf_model'
input_data = tf.constant(1., shape=[1, 1])
to_save = root.f.get_concrete_function(input_data)
tf.saved_model.save(root, saved_model_dir, to_save)

tf_model = tf.saved_model.load(saved_model_dir)
concrete_func = tf_model.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]

W1101 14:02:33.537978 4762860864 deprecation.py:506] From /Volumes/data/venv-py36/lib/python3.6/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1781: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [3]:
# convert model into Core ML format
model = coremltools.converters.tensorflow.convert(
    [concrete_func],
    inputs={'x': (1, 1)},
    outputs=['Identity']
)

assert isinstance(model, coremltools.models.MLModel)

0 assert nodes deleted
['Func/StatefulPartitionedCall/input/_2:0', 'StatefulPartitionedCall/mul/ReadVariableOp:0', 'statefulpartitionedcall_args_1:0', 'Func/StatefulPartitionedCall/input/_3:0', 'StatefulPartitionedCall/mul:0', 'StatefulPartitionedCall/ReadVariableOp:0', 'statefulpartitionedcall_args_2:0']
6 nodes deleted
0 nodes deleted
0 nodes deleted
2 identity nodes deleted
0 disconnected nodes deleted
[SSAConverter] Converting function main ...
[SSAConverter] [1/3] Converting op type: 'Placeholder', name: 'x', output_shape: (1, 1).
[SSAConverter] [2/3] Converting op type: 'Const', name: 'StatefulPartitionedCall/mul'.
[SSAConverter] [3/3] Converting op type: 'Mul', name: 'Identity', output_shape: (1, 1).


## Using Control Flow

In [4]:
# construct a TensorFlow 2.0+ model with tf.function()

@tf.function(input_signature=[tf.TensorSpec([], tf.float32)])
def control_flow(x):
    if x <= 0:
        return 0.
    else:
        return x * 3.

to_save = tf.Module()
to_save.control_flow = control_flow

saved_model_dir = './tf_model'
tf.saved_model.save(to_save, saved_model_dir)
tf_model = tf.saved_model.load(saved_model_dir)
concrete_func = tf_model.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]

In [5]:
# convert model into Core ML format
model = coremltools.converters.tensorflow.convert(
    [concrete_func],
    inputs={'x': (1,)},
    outputs=['Identity']
)

assert isinstance(model, coremltools.models.MLModel)

0 assert nodes deleted
['PartitionedCall/cond/then/_2/Identity_1:0', 'PartitionedCall/LessEqual/y:0', 'PartitionedCall/cond/else/_3/mul/y:0', 'Func/PartitionedCall/cond/then/_2/output/_14:0', 'PartitionedCall/cond/then/_2/Const_1:0']
2 nodes deleted
Fixing cond at merge location: PartitionedCall/cond/output/_9
In an IFF node  fp32  !=  tensor[fp32,1]
0 nodes deleted
0 nodes deleted
2 identity nodes deleted
0 disconnected nodes deleted
[SSAConverter] Converting function main ...
[SSAConverter] [1/7] Converting op type: 'Placeholder', name: 'x', output_shape: (1,).
[SSAConverter] [2/7] Converting op type: 'Const', name: 'PartitionedCall/LessEqual/y'.
[SSAConverter] [3/7] Converting op type: 'Const', name: 'Func/PartitionedCall/cond/then/_2/output/_14'.
[SSAConverter] [4/7] Converting op type: 'Const', name: 'PartitionedCall/cond/else/_3/mul/y'.
[SSAConverter] [5/7] Converting op type: 'LessEqual', name: 'PartitionedCall/LessEqual', output_shape: (1,).
[SSAConverter] [6/7] Converting op t

In [6]:
# try with some sample inputs

inputs = [-3.7, 6.17, 0.0, 1984., -5.]
for data in inputs:
    out1 = to_save.control_flow(data).numpy()
    out2 = model.predict({'x': np.array([data])})['Identity']
    np.testing.assert_array_almost_equal(out1, out2)

## Using `tf.keras` Subclassing APIs

In [7]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(4)
        self.dense2 = tf.keras.layers.Dense(5)

    @tf.function
    def call(self, input_data):
        return self.dense2(self.dense1(input_data))

keras_model = MyModel()

In [8]:
inputs = np.random.rand(4, 4)

# subclassed model can only be saved as SavedModel format
keras_model._set_inputs(inputs)
saved_model_dir = './tf_model_subclassing'
keras_model.save(saved_model_dir, save_format='tf')
# convert and validate
model = coremltools.converters.tensorflow.convert(
    saved_model_dir,
    inputs={'input_1': (4, 4)},
    outputs=['Identity']
)
assert isinstance(model, coremltools.models.MLModel)
# verify the prediction matches
keras_prediction = keras_model.predict(inputs)
prediction = model.predict({'input_1': inputs})['Identity']
np.testing.assert_array_equal(keras_prediction.shape, prediction.shape)
np.testing.assert_almost_equal(keras_prediction.flatten(), prediction.flatten(), decimal=4)

0 assert nodes deleted
['my_model/StatefulPartitionedCall/args_3:0', 'Func/my_model/StatefulPartitionedCall/input/_2:0', 'Func/my_model/StatefulPartitionedCall/StatefulPartitionedCall/input/_11:0', 'my_model/StatefulPartitionedCall/args_4:0', 'Func/my_model/StatefulPartitionedCall/input/_4:0', 'Func/my_model/StatefulPartitionedCall/StatefulPartitionedCall/input/_12:0', 'my_model/StatefulPartitionedCall/args_2:0', 'my_model/StatefulPartitionedCall/StatefulPartitionedCall/dense_1/StatefulPartitionedCall/MatMul/ReadVariableOp:0', 'Func/my_model/StatefulPartitionedCall/StatefulPartitionedCall/dense_1/StatefulPartitionedCall/input/_25:0', 'Func/my_model/StatefulPartitionedCall/input/_3:0', 'Func/my_model/StatefulPartitionedCall/StatefulPartitionedCall/input/_13:0', 'Func/my_model/StatefulPartitionedCall/input/_5:0', 'Func/my_model/StatefulPartitionedCall/StatefulPartitionedCall/input/_10:0', 'Func/my_model/StatefulPartitionedCall/StatefulPartitionedCall/dense_1/StatefulPartitionedCall/input